In [1]:
!pip install tokenizers==0.5.2

     |████████████████████████████████| 3.7MB 37.4MB/s 


In [2]:
!pip install simpletransformers

     |████████████████████████████████| 163kB 21.3MB/s 
     |████████████████████████████████| 204kB 43.4MB/s 
     |████████████████████████████████| 573kB 50.0MB/s 
     |████████████████████████████████| 1.0MB 41.1MB/s 
     |████████████████████████████████| 890kB 45.7MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=33ca2eb501ab2602862791e755bdfe8802141778f7cfd225fa02f11186621ea9
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=8c84d1ba8ddfecedca74a5159bc7f45e8ee16638b7784fac20d8a46948761ce0
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built seqeval sacremoses


In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive/')

In [0]:
import os
import re
import pandas as pd
import numpy as np

In [0]:
def random_seed(seed_value):
    import random 
    random.seed(seed_value) # Python
    import numpy as np
    np.random.seed(seed_value) # cpu vars
    import torch
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

In [0]:
random_seed(42)

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/Competitions/Zindi_VaccinateOrNot/Train.csv')
test = pd.read_csv('/content/gdrive/My Drive/Competitions/Zindi_VaccinateOrNot/Test.csv')
sub = pd.read_csv('/content/gdrive/My Drive/Competitions/Zindi_VaccinateOrNot/SampleSubmission.csv')

In [49]:
train['label'][~train['label'].isin([0, -1, 1])] = -1
train = train.dropna()
test['safe_text'] = test['safe_text'].fillna('xxxxxx')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
train['label'] = train['label'] * train['agreement']
train = train[['safe_text','label']]
test = test[['safe_text']]

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_cv =  train_test_split(train, test_size=0.20, random_state=5, stratify=train['label'])

In [0]:
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import accuracy_score, log_loss, mean_squared_error

In [0]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=1, use_cuda=True, 
                            args={'fp16': False,
                                  'learning_rate': 4e-5,
                                  'regression': True, 
                                  'overwrite_output_dir': True, 
                                  'num_train_epochs': 3,
                                  'manual_seed': 0
                                  }
                            )

In [67]:
model.train_model(X_train)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:247: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 0.248024

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.212502


Running loss: 0.290293


Running loss: 0.268203



In [68]:
scores, model_outputs, wrong_predictions = model.eval_model(X_cv)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:664: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [69]:
scores

{'eval_loss': 0.18572730297688395}

In [70]:
np.sqrt(mean_squared_error(X_cv['label'], model_outputs))

0.43096090507214435

In [71]:
predictions, raw_output = model.predict(test['safe_text'])

In [72]:
raw_output.shape

(5177,)

In [73]:
sub['label'] = raw_output
sub.head(3)

,tweet_id,label
0,00BHHHP1,0.860478
1,00UNMD0E,0.551548
2,01AXPTJF,-0.065648


In [74]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(sub)